In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import json

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os,boto3,sys,glob,json

### Part 1  Loading the  data 

#### Read in csv file

In [2]:
testDF=pd.read_csv('run1000charts_0521.csv')
#testDF= pd.read_csv('/home/jovyan/work/Analytics_Data_training/bfAI_allinst_7502charts_061019.csv') # 7502 charts

# the csv is generated in D drive D:/chartai_qa  with extract1000charts_jsontxt.py, then uploaded to here

In [3]:
len(testDF)

76792

In [4]:
testDF.columns

Index(['Unnamed: 0', 'chart_id', 'code', 'text', 'start_offset', 'end_offset',
       'text75'],
      dtype='object')

In [5]:
testDF=testDF.drop(['Unnamed: 0'],axis=1)

In [6]:
testDF.head()

,chart_id,code,text,start_offset,end_offset,text75
0,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E063,Ht,1187,1189,ocedure codes 36415 venipunct routine vital si...
1,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,1374,1388,tion list reviewed reconciled patient past med...
2,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,1391,1403,reconciled patient past medical history hyperl...
3,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,E785,Hyperlipidemia,2957,2971,tion list reviewed reconciled patient past med...
4,IBXRMA2018_MULT_1163297620010001_HMK_179342168...,I10,Hypertension,2974,2986,reconciled patient past medical history hyperl...


In [7]:
# just check if there's null text, if yes, remove them 
nulltext=testDF[~pd.notnull(testDF['text'])] 
len(nulltext)

0

In [8]:
nulltext=testDF[~pd.notnull(testDF['text75'])] 
len(nulltext)

26

In [9]:
nulltext   # there are some text not correctly extracted

,chart_id,code,text,start_offset,end_offset,text75
1336,IBXRMA2018_MULT_1164353980010001_HMK_160303709...,G129,SMA,4,7,NaN
2198,IBXRMA2018_MULT_1176161080010001_HMK_192365603...,N08,Kidney,46,52,NaN
3876,IBXRMA2018_MULT_1202740400010001_HMK_153425750...,G129,SMA,4,7,NaN
4032,IBXRMA2018_MULT_1202908350010001_HMK_210648786...,E744,PC,25,27,NaN
5591,IBXRMA2018_MULT_1210779380010001_HMK_184462858...,G129,SMA,4,7,NaN
6465,IBXRMA2018_MULT_1212745310010001_HMK_165441851...,N08,Kidney,48,54,NaN
9116,IBXRMA2018_MULT_1216729130010001_HMK_196286386...,N08,Kidney,46,52,NaN
14970,IBXRMA2018_MULT_1216856710010001_HMK_173309591...,G129,SMA,4,7,NaN
18950,IBXRMA2018_MULT_1216874020010001_HMK_143364782...,E744,PC,25,27,NaN
21748,IBXRMA2018_MULT_1216901800010001_HMK_201224196...,G129,SMA,4,7,NaN


In [10]:
testDF=testDF[pd.notnull(testDF['text75'])] 
len(testDF)

76766

In [11]:
testDF['chart_id'].nunique()   # only 980 charts not 1044 charts

980

In [13]:
testDF=testDF.reset_index(drop=True)   # very very import, to reset the index, otherwise the below running the model will not be right, because I use the index as reference for position and matching them
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...


### Part 2 Loading all the models or locate the directory

#### Part 2.a. Load the 28 pickle files and run them on the testDF with selected chart_ids

In [14]:
#files=glob.glob("../../picklefolder_ngrams/refreshed_models_update0531/*.pickle") # refreshed updated models
files=glob.glob("../../picklefolder_ngrams/refreshed_models/*19.pickle")  # refreshed models
#files=glob.glob("../../picklefolder_ngrams/pipeline_pickles/*.pickle")  # pipeline models

In [15]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [16]:
#files=files[:-1]
#len(files)

In [17]:
files2=glob.glob("../../picklefolder_ngrams/refreshed_models_update0531/*.pickle") # refreshed updated models
#files2=files2[:-1]   # run 28 model
files2=files2[28]  

In [18]:
files.append(files2)
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [19]:
files[27].split('/')[-1].split('_')[1]

'preventative'

In [20]:
files[10].split('/')[-1]

'Code_Heart_LRApril0219.pickle'

In [21]:
files[24]

'../../picklefolder_ngrams/refreshed_models/DeletionReason_not_doctors_note_LRApril0219.pickle'

### Part 3. Apply models to test data

In [22]:
#encoder = preprocessing.LabelEncoder()
#test_y = encoder.fit_transform(testDF['flag'])  


In [23]:
code_list={}
code_list["Model1"]=['I10','E119']
code_list["Model2"]=['I2510','E785','K219','F329','I639']
code_list["Model3"]=['I739','N183','Z992','B20','I213','R569','I43','J410','I714','R579','E550','I209','J45998','I480','B182','K210','K739']
code_list["Heart"]=['I509','I482','I4891','I82401']
code_list["Depression"]=['F411','F410','F40240','F458','F409','F4000','F4001','F445','F444','F40243','F446','F4010','F442','F408','F449','F451']
code_list["Kidney"]=['N189','N181','N182','E1122','I130','E0822','E1022']
code_list["Diabetes"]=['Z794','E109','E119','E139','E089','E099']
code_list["Cancer"]=['C801','C50919','C189']
code_list["COPD"]=['J449','J45909']
code_list["Cholesterol"]=['E780','E785','E782','E789','E7800','E784','E781','E7801','E7881','E882','E783','E786','E7889','E756','E755','E7130']

In [24]:
allpredict=pd.DataFrame()   # create an empty dataframe, to save the prediction result for each model
allprob=pd.DataFrame()
print(allprob)

Empty DataFrame
Columns: []
Index: []


In [25]:
# for one model evaluation
'''
indexdf=pd.DataFrame() 
model_pipeline = joblib.load(files)
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model

testDF1=testDF
indexdf[0]= [1 for i in range(len(testDF))]
        
        
        #print(files[i])
    # how to put the prediction on the right index?
test=testDF1['text75']
print("model loaded:", files )
print("length of data for the model:", len(test))
    
   
proba = model_pipeline.predict_proba(test)[:,1]
prediction = np.where(proba > 0.4, 1, 0) # use a lower threshold
testDF1['key']=testDF1.index
testDF1['pred']=prediction
testDF1['prob']= proba

indexdf['key']=indexdf.index
merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
allpredict[0]=merged['pred']    # save the prediction to corresponding dataframe
allprob[0]=merged['prob']
'''

'\nindexdf=pd.DataFrame() \nmodel_pipeline = joblib.load(files)\n  #  f = open("allmodeltestscore_0506.txt", "a+")\n    # deletion model\n\ntestDF1=testDF\nindexdf[0]= [1 for i in range(len(testDF))]\n        \n        \n        #print(files[i])\n    # how to put the prediction on the right index?\ntest=testDF1[\'text75\']\nprint("model loaded:", files )\nprint("length of data for the model:", len(test))\n    \n   \nproba = model_pipeline.predict_proba(test)[:,1]\nprediction = np.where(proba > 0.4, 1, 0) # use a lower threshold\ntestDF1[\'key\']=testDF1.index\ntestDF1[\'pred\']=prediction\ntestDF1[\'prob\']= proba\n\nindexdf[\'key\']=indexdf.index\nmerged=pd.merge(indexdf,testDF1,on=\'key\',how=\'outer\')  # maintain the position of the perdiction, match to the right index\nallpredict[0]=merged[\'pred\']    # save the prediction to corresponding dataframe\nallprob[0]=merged[\'prob\']\n'

In [26]:
#f = open("allmodeltestscore_0506.txt", "w+")  # save all the score to this file
# for multiple model evaluation

   # create a dataframe, otherwise can use testDF for merging to obtain the prediction matched to index
# because for some model testing, we only select partial data, like phrase-based model, code-based model
for i in range(len(files)):
    indexdf=pd.DataFrame() 
    model_pipeline = joblib.load(files[i])
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model
    if(files[i].split('/')[-1].startswith('D')):
        try:
            testDF1=testDF
            indexdf[i]= [1 for i in range(len(testDF))]
        except:
            print('no data for this model')
        
    elif(files[i].split('/')[-1].startswith('P')):
    # if it's phrase-based model, use the following way to select data
        
        phrase=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['text'].str.contains(phrase,flags=re.IGNORECASE)]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')

    elif(files[i].split('/')[-1].startswith('C')):
    # if it's code based model, use the following way to select model
        codereason=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['code'].isin(code_list[codereason])]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')
    else:
        print('not a valid/good model name probably...')

    print(files[i])
    # how to put the prediction on the right index?
    test=testDF1['text75']
    print("length of data for the model:", len(test))
    try:
        
        proba = model_pipeline.predict_proba(test)[:,1]
        prediction = np.where(proba > 0.4, 1, 0) # use a lower threshold
        #prediction = model_pipeline.predict(test)
        
        
        testDF1['key']=testDF1.index
        testDF1['pred']=prediction
        testDF1['prob']= proba

        indexdf['key']=indexdf.index
        merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
        allpredict[i]=merged['pred']    # save the prediction to corresponding dataframe
        allprob[i]=merged['prob']   # save the probability
    except:
         print('something is wrong')


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle
length of data for the model: 167


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle
length of data for the model: 183


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

...

../../picklefolder_ngrams/refreshed_models/DeletionReason_not_relevant_LRApril0219.pickle
length of data for the model: 76766


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/DeletionReason_phoneORaddressORother_number_LRApril0219.pickle
length of data for the model: 76766


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

../../picklefolder_ngrams/refreshed_models/DeletionReason_preventative_screening_LRApril0219.pickle
length of data for the model: 76766
../../picklefolder_ngrams/refreshed_models_update0531/Deletion_Consolidated_LRMay2919.pickle
length of data for the model: 76766


####  Part 2.b. Now get the prediction and probability dataframes and merge with testDF which has all the chart_id etc info

In [27]:
len(allpredict)

76766

In [28]:
allpredict.isnull().sum()  # a lot of nulls because not all data are run for each model

0     76599
1     76583
2     69367
3     76248
4     73545
5     75332
6     76185
7     74772
8     75576
9     70135
10    73588
11    75610
12    60476
13    75301
14    68037
15    65222
16    70621
17    76002
18    74736
19    76184
20    75238
21    71545
22        0
23        0
24        0
25        0
26        0
27        0
28        0
dtype: int64

In [29]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [30]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014626,0.063301,0.160061,0.349433,0.196206,0.026679,0.521088
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.005715,0.021906,0.020609,0.160427,0.018062,0.008571,0.197786
2,NaN,NaN,0.113764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.008170,0.036722,0.046403,0.221087,0.024644,0.018220,0.331177
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.002357,0.006155,0.004079,0.047251,0.002172,0.002319,0.108050
4,NaN,NaN,0.125394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.003010,0.007203,0.021988,0.101171,0.002850,0.004171,0.253157


In [31]:
len(allprob)

76766

In [32]:
len(testDF)

76766

In [33]:
# the data frame for predictions from each model
#merge to testDF to keep the Nan, which means not omputed for that instance
#testDF2=pd.concat([testDF,allpredict],axis=1)
#len(testDF2)
testDF['index']=testDF.index
allpredict['index']=allpredict.index
df_pred = pd.merge(testDF,allpredict, how='left', on = 'index')
len(df_pred)

76766

In [34]:
df_pred.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,19,20,21,22,23,24,25,26,27,28
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,76761,1,0.693213,76761,...,NaN,NaN,NaN,0,0,0,0,0,0,1
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,76762,0,0.296027,76762,...,NaN,NaN,NaN,0,0,0,0,0,0,0
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,76763,0,0.246400,76763,...,NaN,NaN,NaN,0,0,0,0,0,1,0
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,76764,0,0.178136,76764,...,NaN,NaN,NaN,0,0,0,0,0,0,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,76765,1,0.899252,76765,...,NaN,NaN,NaN,0,0,1,1,0,1,1


In [35]:
# now the data frame with probabilities
testDF['index']=testDF.index
allprob['index']=allprob.index
df_prob = pd.merge(testDF,allprob, how='left', on = 'index')
print(len(df_prob))

76766


In [36]:
df_prob.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,19,20,21,22,23,24,25,26,27,28
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,76761,1,0.693213,76761,...,NaN,NaN,NaN,0.272709,0.064432,0.027841,0.217822,0.013272,0.018055,0.693213
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,76762,0,0.296027,76762,...,NaN,NaN,NaN,0.012492,0.072086,0.181318,0.381113,0.030548,0.306477,0.296027
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,76763,0,0.246400,76763,...,NaN,NaN,NaN,0.018394,0.025507,0.030261,0.122995,0.003520,0.686125,0.246400
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,76764,0,0.178136,76764,...,NaN,NaN,NaN,0.011497,0.027704,0.075676,0.183114,0.005224,0.194581,0.178136
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,76765,1,0.899252,76765,...,NaN,NaN,NaN,0.154583,0.207874,0.571299,0.816210,0.020743,0.990072,0.899252


#### Part 2.c. Now combine the prediction from 28 models to make one prediction,it's actually an or operation, but here using sum()

In [37]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.014626,0.063301,0.160061,0.349433,0.196206,0.026679,0.521088,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.005715,0.021906,0.020609,0.160427,0.018062,0.008571,0.197786,1
2,NaN,NaN,0.113764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.008170,0.036722,0.046403,0.221087,0.024644,0.018220,0.331177,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.002357,0.006155,0.004079,0.047251,0.002172,0.002319,0.108050,3
4,NaN,NaN,0.125394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.003010,0.007203,0.021988,0.101171,0.002850,0.004171,0.253157,4


In [38]:
allpredictNan=allpredict   # keep the original NaN if that row is not computed for that model
allpredict=allpredict.fillna(0) # fill the NaNs by zero

In [39]:
allpredictNan.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,1,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,1
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,3
4,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,4


In [40]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,4


In [41]:
allpredict.drop(['index'],inplace=True, axis=1)
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [42]:
#allpredict=allpredict[[22,23,24,25,26,27]]  # only select deletion reason columns
#allpredict.columns
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [43]:
allpredict['Sum'] = allpredict.sum(axis = 1, skipna = True) 
df_pred['all_model_pred']=[1 if allpredict['Sum'].iloc[i]>=1 else 0 for i in range(len(allpredict))] 

In [44]:
# check how many 1s are predicted 
allpredict1=df_pred['all_model_pred']
print(sum(allpredict1))
print(len(allpredict1))  # 228562 are predicted as deleted out of 418310

39345
76766


In [45]:
# drop the key and predict prob temp columns
df_pred.drop(['key','pred','prob','index'],inplace=True, axis=1)
df_prob.drop(['key','pred','prob','index'],inplace=True, axis=1)

In [46]:
df_pred.tail()
#all_model_pred is check any model output is 1 for that instance

,chart_id,code,text,start_offset,end_offset,text75,0,1,2,3,...,20,21,22,23,24,25,26,27,28,all_model_pred
76761,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E119,dm2,5902,5905,onciled patient past medical history cholecyst...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,1,1
76762,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,B182,hepatitis c,6819,6830,lab comp metabolic panel lab glycosolated hemo...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,1
76763,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E550,vitamin d deficiency,6951,6971,increat random colon cancer screening notes de...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,1,0,1
76764,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,E782,hyperlipidemia,7001,7015,declined gi referral vitamin deficiency lab vi...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0
76765,IBXRMA2018_MULT_1231614110010001_HMK_196526763...,I2510,cad,7144,7147,breast cancer screening imaging mammo digital ...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,1,1,0,1,1,1


In [47]:
df_pred.iloc[135]   # just check one instance

chart_id          IBXRMA2018_MULT_1163518440010001_HMK_188344641...
code                                                            I10
text                                                            i10
start_offset                                                   1241
end_offset                                                     1244
text75            clerotic heart disease native coronary artery ...
0                                                               NaN
1                                                               NaN
2                                                               NaN
3                                                               NaN
4                                                               NaN
5                                                               NaN
6                                                               NaN
7                                                               NaN
8                                               

In [48]:
df_pred.columns

Index([      'chart_id',           'code',           'text',   'start_offset',
           'end_offset',         'text75',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27,               28, 'all_model_pred'],
      dtype='object')

In [49]:
df_prob.columns

Index([    'chart_id',         'code',         'text', 'start_offset',
         'end_offset',       'text75',              0,              1,
                    2,              3,              4,              5,
                    6,              7,              8,              9,
                   10,             11,             12,             13,
                   14,             15,             16,             17,
                   18,             19,             20,             21,
                   22,             23,             24,             25,
                   26,             27,             28],
      dtype='object')

In [50]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [51]:
'''
# if running refreshed model, use this rename
# rename the columns names, has to be consistent with the above order
# since there's no 0, or 1, 17, these models didn't run because no data for them

col_dict = {0: 'Phrase_stroke', 1: 'Phrase_emphysema',2:'Phrase_hypertension',3:'Phrase_dementia',4:'Phrase_cholesterol',
           5:'Phrase_cad',6:'Phrase_aneurysm',7:'Code_COPD',8:'Code_Depression',9:'Code_Diabetes',10:'Code_Heart',
           11:'Code_Kidney',12:'Code_Model1',13:'Code_Cancer',14:'Code_Cholesterol',15:'Code_Model2',
           16:'Code_Model3',17:'Phrase_asthma',18:'Phrase_cancer',19:'Phrase_ckd',20:'Phrase_depression',21:'Phrase_diabetes',
            22:'DeletionReason_family_history',23:'DeletionReason_negation',24:'DeletionReason_not_doctors_note',
           25:'DeletionReason_not_relevant',26:'DeletionReason_phoneORaddressORother_number',27:'DeletionReason_preventative_screening',
           }   ## key→old name, value→new name

df_pred.columns = [col_dict.get(x, x) for x in df_pred.columns]
df_prob.columns = [col_dict.get(x, x) for x in df_prob.columns]
'''

"\n# if running refreshed model, use this rename\n# rename the columns names, has to be consistent with the above order\n# since there's no 0, or 1, 17, these models didn't run because no data for them\n\ncol_dict = {0: 'Phrase_stroke', 1: 'Phrase_emphysema',2:'Phrase_hypertension',3:'Phrase_dementia',4:'Phrase_cholesterol',\n           5:'Phrase_cad',6:'Phrase_aneurysm',7:'Code_COPD',8:'Code_Depression',9:'Code_Diabetes',10:'Code_Heart',\n           11:'Code_Kidney',12:'Code_Model1',13:'Code_Cancer',14:'Code_Cholesterol',15:'Code_Model2',\n           16:'Code_Model3',17:'Phrase_asthma',18:'Phrase_cancer',19:'Phrase_ckd',20:'Phrase_depression',21:'Phrase_diabetes',\n            22:'DeletionReason_family_history',23:'DeletionReason_negation',24:'DeletionReason_not_doctors_note',\n           25:'DeletionReason_not_relevant',26:'DeletionReason_phoneORaddressORother_number',27:'DeletionReason_preventative_screening',\n           }   ## key→old name, value→new name\n\ndf_pred.columns = [col

In [52]:
# if running pipeline models, use this rename
# rename the columns names, has to be consistent with the above order  
# since there's no 0, or 1, 17, these models didn't run because no data for them
 ## key→old name, value→new name
col_dict = {0:'Code_Cholesterol' , 1: 'DeletionReason_family_history',2:'DeletionReason_negation',3:'DeletionReason_not_doctors_note',4:'Code_Model2',
           5:'DeletionReason_not_relevant',6:'Phrase_stroke',7:'Phrase_hypertension',8:'Phrase_emphysema',9:'Phrase_diabetes',10:'Phrase_depression',
           11:'Phrase_dementia',12:'Phrase_ckd',13:'Phrase_cholesterol',14:'Phrase_cancer',15:'Phrase_cad',
           16:'Phrase_asthma',17:'Phrase_aneurysm',18:'DeletionReason_preventative_screening',19:'DeletionReason_phoneORaddressORother_number',20:'Code_Model3',21:'Code_Model1',
            22:'Code_Kidney',23:'Code_Heart',24:'Code_Diabetes',
           25:'Code_Depression',26:'Code_COPD',27:'Code_Cancer', 28:'consolidated_LR'
           }  

df_pred.columns = [col_dict.get(x, x) for x in df_pred.columns]
df_prob.columns = [col_dict.get(x, x) for x in df_prob.columns]

In [53]:
df_pred.columns

Index([                                   'chart_id',
                                              'code',
                                              'text',
                                      'start_offset',
                                        'end_offset',
                                            'text75',
                                  'Code_Cholesterol',
                     'DeletionReason_family_history',
                           'DeletionReason_negation',
                   'DeletionReason_not_doctors_note',
                                       'Code_Model2',
                       'DeletionReason_not_relevant',
                                     'Phrase_stroke',
                               'Phrase_hypertension',
                                  'Phrase_emphysema',
                                   'Phrase_diabetes',
                                 'Phrase_depression',
                                   'Phrase_dementia',
                            

In [55]:
#df_pred.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_pred[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()

In [56]:
#df_prob.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_prob[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()  #ok make sense

In [57]:
#df_pred.to_csv('run1000charts_pred_0521.csv')   #  with prediction for each model and all model combined
#df_prob.to_csv('probdataforcompwithAM_0506.csv')   #  with probability for each model
# finally corrected a lot of errors like index not reset issue

In [58]:
print(len(set(df_pred['chart_id'].tolist())))
print(len(set(df_prob['chart_id'].tolist())))   # 813 unique chart ids

980
980


#### Part 2.d. Load LSTM model and evaluate

In [102]:
# load modules
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [103]:
with open("../../picklefolder_ngrams/temp_4deletionmodels_refreshed/keras_tokenizer_embeddings_0328.pickle", "rb") as f:
   tokenizer = pickle.load(f)    # lOAD tokenizer

In [104]:
max_len = 70

In [105]:
print(len(testDF))
# tokenize the test dataset
sequences = tokenizer.texts_to_sequences(testDF['text75'].values)
#word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))
testX1 = pad_sequences(sequences, maxlen=max_len)
#y_test1 = testdf['flag']
print(testX1.shape)  # ,y_test1.shape)    # no flag for the data

76766
(76766, 70)


In [106]:
# load the lstm model
model2=joblib.load('../../picklefolder_ngrams/temp_4deletionmodels_refreshed/DeleteReason_biclass_Mar2819_lstm.pickle')

W0624 19:41:58.744635 139737363326784 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0624 19:41:58.758430 139737363326784 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 19:41:58.775067 139737363326784 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0624 19:41:58.775788 139737363326784 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0624 19:41:58.925335

In [107]:
predictiont2 = model2.predict(testX1)

finallabelt1= (predictiont2 > 0.4).astype(np.int)    # using different threshold
#finallabelt2= (predictiont2 > 0.3).astype(np.int) 
#finallabelt3= (predictiont2 > 0.2).astype(np.int)

###  Part 3. Load in coder feedback and  evaluate the accuracy

#### Select common chart_ids from both 1000 model run and 6000 coder annotation

In [59]:
df_pred.columns

Index([                                   'chart_id',
                                              'code',
                                              'text',
                                      'start_offset',
                                        'end_offset',
                                            'text75',
                                  'Code_Cholesterol',
                     'DeletionReason_family_history',
                           'DeletionReason_negation',
                   'DeletionReason_not_doctors_note',
                                       'Code_Model2',
                       'DeletionReason_not_relevant',
                                     'Phrase_stroke',
                               'Phrase_hypertension',
                                  'Phrase_emphysema',
                                   'Phrase_diabetes',
                                 'Phrase_depression',
                                   'Phrase_dementia',
                            

In [108]:
df_pred['lstm_red']=finallabelt1

In [110]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,Sum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,1,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0.0


In [300]:
allpredict.columns

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
          10,    11,    12,    13,    14,    15,    16,    17,    18,    19,
          20,    21,    22,    23,    24,    25,    26,    27,    28, 'Sum'],
      dtype='object')

In [301]:
allpredict_28=allpredict[[0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
          10,    11,    12,    13,    14,    15,    16,    17,    18,    19,
          20,    21,    22,    23,    24,    25,    26,    27, 28]]
allpredict_28.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [302]:
allpredict_28['lstm']=finallabelt1

In [303]:
allpredict_28.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,lstm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0


In [304]:
allpredict_28['Sum'] = allpredict_28.sum(axis = 1, skipna = True) 
df_pred['all_model_pred']=[1 if allpredict_28['Sum'].iloc[i]>=1 else 0 for i in range(len(allpredict))] 

In [305]:
allpredict1=df_pred['all_model_pred']
print(sum(allpredict1))
print(len(allpredict1)) 

41963
76766


In [306]:
#df_pred['all_model_pred']=df_pred['DeletionReason_not_relevant']
#model_name='Code_Cholesterol'
#df_pred['all_model_pred']=df_pred['DeletionReason_family_history']

In [307]:
# model computation
df_pred1=df_pred[['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75','all_model_pred']]
print(len(df_pred1))
df_pred1 = df_pred1.drop_duplicates()
print(len(df_pred1))

76766
76766


In [308]:
# read in all charts with coder feedback
df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_with_chartids_29042019.csv') # 1000 charts
#df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_output_withchart_id_100619.csv') #7500 chart
print(len(df_codeall))

136527


In [309]:
print("unique chart is in 1000 charts run model output:",df_pred1['chart_id'].nunique())
print("unique chart is in 6000 charts with coder feedback:",df_codeall['chart_id'].nunique())

unique chart is in 1000 charts run model output: 980
unique chart is in 6000 charts with coder feedback: 5982


In [310]:
# now find overlap chart ids
overlapped_charts = pd.merge(df_codeall[['chart_id']].drop_duplicates(),df_pred1[['chart_id']].drop_duplicates(), how = 'inner')
overlapped_charts = list(overlapped_charts['chart_id'].unique())
print(len(overlapped_charts))  

798


In [311]:
# select the model output
fd_prod = df_pred1[['chart_id','code','start_offset','end_offset','text75','all_model_pred']][df_pred1['chart_id'].isin(overlapped_charts)].drop_duplicates()
len(fd_prod)

60131

In [312]:
# select the model output, if it's individual model, need to select those not NaN instances only
fd_prod.dropna(inplace=True)
print(len(fd_prod))
#fd_prod.isnull().sum()

60131


In [313]:
# select those in the code output
data = df_codeall[df_codeall['chart_id'].isin(overlapped_charts)]
print(len(data))
print(len(data[data['label'] == 'added']))
print(len(data[data['label'] == 'deleted']))
print(len(data[data['label'] == 'agreed']))

30057
17974
3232
8851


In [314]:
# The pipeline TN can be computed as, the following is not true negative yet
TNh= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))
print("TNh:",TNh)
TNh= len(fd_prod)-(len(data)-len(data[data['label'] == 'deleted']))
print("TNh:",TNh)

TNh: 48048
TNh: 33306


#### The accurary number for the pipeline models

In [315]:
# Use fuzzy logic to compute the deleted, added, agreed
data = data[data['start'] != 0]
data = data[data['end'] != 0]
#data = data[data['created_at'] == date ] ##Change date to filter for datewise data
#label_counts = data.groupby(['label']).size().reset_index(name='counts')
instmatches = data[data['label'] == 'agreed']
instfuzzymatches = data[data['label'] == 'deleted']
instfuzzymatches = instfuzzymatches[instfuzzymatches['deleted_reason'].isin(['Incorrect Specification - Non-Risk Adjusted','repeated_instance','incorrect_year_of_service']) ]
instmatches = pd.concat([instmatches, instfuzzymatches])
instmatches = instmatches[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

instdeleted = data[data['label'] == 'deleted']
instdeleted = instdeleted[instdeleted['deleted_reason'] != 'other']
instdeleted = pd.merge(instdeleted, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instdeleted = instdeleted[instdeleted['_merge'] == 'left_only']
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end','deleted_reason']].drop_duplicates()
deleted_reason = instdeleted.groupby(['deleted_reason']).size().reset_index(name = 'counts')
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end']].drop_duplicates()


instadded = data[data['label'] == 'added']
instadded = pd.merge(instadded, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()
instadded = pd.merge(instadded, instdeleted,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)
print(added)
print(deleted)
print(agreed)

16376
770
9168


In [316]:
instmatches.columns

Index(['chart_id', 'submission_id', 'code_id', 'start', 'end'], dtype='object')

In [317]:
# my added small part
instmatchessub=instmatches[['chart_id', 'code_id', 'submission_id','start','end']]
instmatchessub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']

In [318]:
len(fd_prod)

60131

In [319]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
prod_additions_models = pd.merge(instadded, fd_prod,how = 'inner', on = ['chart_id'], indicator = True)
prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
prod_additions_models1 = prod_additions_models[ (
                    (prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_start'] <= 75))
          | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
          | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
          |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_end'] <= 75))
          ]
instadded_modeldeleted = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()

In [320]:
len(instadded_modeldeleted)

15514

In [321]:
# initial numbers considering all the additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded)
added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))


precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 9168
TN: 48048
FN: 16376
FP: 770
precision: 0.9225196216542564
recall: 0.3589101158784842
accuracy: 0.7694252440762755
-------------------
Agreements =  9168
Deletions =  770
Additions =  16376


In [322]:
# initial numbers considering model deleted additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded_modeldeleted)
added = len(instadded_modeldeleted)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN1= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN1+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)


TP: 9168
TN: 48048
FN: 15514
FP: 770
precision: 0.9225196216542564
recall: 0.37144477757069927
accuracy: 0.7784489795918368
-------------------
Agreements =  9168
Deletions =  770
Additions =  15514


####  Evaluated   Model accuracy numbers

In [323]:
print(len(df_pred1))   # model output with all the instances

76766


In [324]:
print(len(fd_prod))  # already selected  the 798 chart  from previous steps

60131


In [325]:
fd_prod.dtypes

chart_id          object
code              object
start_offset       int64
end_offset         int64
text75            object
all_model_pred     int64
dtype: object

In [326]:
TN=len(fd_prod[fd_prod['all_model_pred']==1])  # predicted deleted instances, but not true negative
print("TN:",TN)

TN: 32100


In [327]:
# select those 708 charts and only passed the filtering not deleted, 0 is keep here

dev_df_sub_1 = fd_prod[fd_prod['all_model_pred']==0]  # select the 798 chart
len(dev_df_sub_1[['chart_id','start_offset']].drop_duplicates())  # 37085 pass through

28031

In [328]:
dev_df_sub_1.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)

In [329]:
dev_df_sub_1.rename(columns={'code':'code_id'},inplace=True)

In [330]:
new_matches = pd.merge(dev_df_sub_1[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
new_deleted = pd.merge(dev_df_sub_1[['chart_id','start']], instdeleted,how = 'inner', on = ['chart_id','start'], indicator = True)

In [331]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
new_added = pd.merge(dev_df_sub_1, instadded,how = 'inner', on = ['chart_id'], indicator = True)
new_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
new_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])

In [332]:
new_added1 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 5))
          | (new_added['start_x'] == new_added['start_y'])
          ]

In [333]:
#1. if the start offset matches exactly
#2. If the end offset matches exactly
#3. If the overlap of start offset is between 75 characters and the code id matches
#4. If the overlap of end offset is between 75 characters and the code id matches
new_added2 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 75))
          | (new_added['start_x'] == new_added['start_y'])
          | (new_added['end_x'] == new_added['end_y'])
          |((new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_end'] <= 75))
          ]

In [334]:
instadded_modeldeleted.columns

Index(['chart_id', 'submission_id', 'code', 'start_offset_y', 'end_offset_y'], dtype='object')

In [335]:
#just validating the additions with a different method of calculation
instadded_modeldeleted.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
x = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
             , instadded_modeldeleted[['chart_id','start']].drop_duplicates()
             ,how = 'inner', on = ['chart_id','start'], indicator = True)
len(x)

11673

In [336]:
print(len(new_matches))
print(len(new_deleted))
print(len(new_added1[['chart_id','start_x']].drop_duplicates()))
print(len(new_added2[['chart_id','start_x']].drop_duplicates()))

8519
572
8434
11673


In [337]:
#type 1 - older number of additions considering only start offset
refreshed_agreements1 = int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions1 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions1 = added - int(len(new_added1[['chart_id','start_x']].drop_duplicates()))

#type 2 - new number of additions considering start offset and end offset
refreshed_agreements2 = int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions2 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions2 = added - int(len(new_added2[['chart_id','start_x']].drop_duplicates()))

print(refreshed_agreements1)
print(refreshed_deletions1)
print(refreshed_additions1)
print("--------")
print(refreshed_agreements2)
print(refreshed_deletions2)
print(refreshed_additions2)

16953
11078
7080
--------
20192
7839
3841


In [338]:
refreshed_total1 = refreshed_agreements1 + refreshed_deletions1 + refreshed_additions1
refreshed_total2 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions2
print(refreshed_total1)
print(refreshed_total2)

35111
31872


In [339]:
# my added part of calculating TN
fd_prodsub=fd_prod[fd_prod['all_model_pred']==1]
fd_prodsub2=fd_prod[fd_prod['all_model_pred']==0]
print(len(fd_prodsub))   # predicted as delete
print(len(fd_prodsub2))  # not deleted, presented to coder
fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']   # model predicted as deleted
fd_prodsub2.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']   # model predicted as passing through or agree

32100
28031


In [340]:
instaddedsub=instadded[['chart_id', 'submission_id','code_id', 'start','end']]
instaddedsub.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']


In [341]:
fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']

In [342]:
instmatchedaddedsub = pd.concat([instaddedsub,instmatchessub])
#instmatchedaddedsub=instmatchedaddedsub[['chart_id', 'code_id', 'submission_id','start','end']]
#instmatchedaddedsub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']
#print(len(instmatchedaddedsub))  # 25273
prod_agree_models = pd.merge(instmatchedaddedsub, fd_prodsub,how = 'inner', on = ['chart_id'], indicator = True)
len(prod_agree_models)

prod_agree_models['abs_diff_start'] = abs(prod_agree_models['start_offset_x'] - prod_agree_models['start_offset_y'])
prod_agree_models['abs_diff_end'] = abs(prod_agree_models['end_offset_x'] - prod_agree_models['end_offset_y'])

prod_agree_models1 = prod_agree_models[ (
                    (prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                    & (prod_agree_models['abs_diff_start'] <= 5))
          | (prod_agree_models['start_offset_x'] == prod_agree_models['start_offset_y'])
          | (prod_agree_models['end_offset_x'] == prod_agree_models['end_offset_y'])
          |((prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                    & (prod_agree_models['abs_diff_end'] <= 5))
          ]

instaddagreed_modeldeleted = prod_agree_models1[['chart_id','submission_id','code_id_x','code_id_y','start_offset_x','end_offset_x']].drop_duplicates()
len(instaddagreed_modeldeleted)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


4112

In [343]:
#type 1 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements1*100/refreshed_total1,'%')
print('Deletions = ', refreshed_deletions1*100/refreshed_total1,'%')
print('Additions = ', refreshed_additions1*100/refreshed_total1,'%')
print('Precision = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_deletions1),'%')
print('Recall = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_additions1),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements1)
print('Deletions = ', refreshed_deletions1)
print('Additions = ', refreshed_additions1)

TP = refreshed_agreements1
FP = refreshed_deletions1
FN = refreshed_additions1

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)


Instance level New Models:
Agreements =  48.284013557004926 %
Deletions =  31.5513656688787 %
Additions =  20.164620774116372 %
Precision =  60.47946915914523 %
Recall =  70.54050680314568 %
-------------------
Agreements =  16953
Deletions =  11078
Additions =  7080
TP: 16953
TN: 27988
FN: 7080
FP: 11078
precision: 0.6047946915914523
recall: 0.7054050680314568
accuracy: 0.712229987796954


In [344]:
#type 2 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total2,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total2,'%')
print('Additions = ', refreshed_additions2*100/refreshed_total2,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions2),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions2)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions2

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)



Instance level New Models:
Agreements =  63.35341365461847 %
Deletions =  24.595256024096386 %
Additions =  12.05133032128514 %
Precision =  72.03453319539082 %
Recall =  84.01780884616986 %
-------------------
Agreements =  20192
Deletions =  7839
Additions =  3841
TP: 20192
TN: 27988
FN: 3841
FP: 7839
precision: 0.7203453319539082
recall: 0.8401780884616985
accuracy: 0.8048780487804879


In [345]:
# save a html copy
#!jupyter nbconvert --to html all28refreshedmodels_comptoAMresult_050819-deleteinstances.ipynb